In [1]:
from nepal.datasets import NYTimes, Vaccinations, CountyDistance, GovernmentResponse

In [2]:
# nytimes = NYTimes().load()
# vaccinations = Vaccinations().load()
# distances = CountyDistance(radius=100).load()
response_tracker = GovernmentResponse().load()

In [3]:
response_tracker[response_tracker["CountryName"] == "United States"]

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_combined_numeric,C1_combined,C2_combined_numeric,C2_combined,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
242600,United States,USA,<NA>,<NA>,NAT_TOTAL,2020-01-01,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242601,United States,USA,<NA>,<NA>,NAT_TOTAL,2020-01-02,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242602,United States,USA,<NA>,<NA>,NAT_TOTAL,2020-01-03,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242603,United States,USA,<NA>,<NA>,NAT_TOTAL,2020-01-04,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242604,United States,USA,<NA>,<NA>,NAT_TOTAL,2020-01-05,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288043,United States,USA,Wyoming,US_WY,STATE_TOTAL,2022-05-19,NaN,<NA>,NaN,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288044,United States,USA,Wyoming,US_WY,STATE_TOTAL,2022-05-20,NaN,<NA>,NaN,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288045,United States,USA,Wyoming,US_WY,STATE_TOTAL,2022-05-21,NaN,<NA>,NaN,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288046,United States,USA,Wyoming,US_WY,STATE_TOTAL,2022-05-22,NaN,<NA>,NaN,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
response_tracker.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_combined_numeric', 'C1_combined',
       'C2_combined_numeric', 'C2_combined', 'C3_combined_numeric',
       'C3_combined', 'C4_combined_numeric', 'C4_combined',
       'C5_combined_numeric', 'C5_combined', 'C6_combined_numeric',
       'C6_combined', 'C7_combined_numeric', 'C7_combined',
       'C8_combined_numeric', 'C8_combined', 'E1_combined_numeric',
       'E1_combined', 'E2_combined_numeric', 'E2_combined',
       'H1_combined_numeric', 'H1_combined', 'H2_combined_numeric',
       'H2_combined', 'H3_combined_numeric', 'H3_combined',
       'H6_combined_numeric', 'H6_combined', 'H7_combined_numeric',
       'H7_combined', 'H8_combined_numeric', 'H8_combined',
       'V1_combined_numeric', 'V1_combined', 'V2_combined_numeric',
       'V2_combined', 'V3_combined_numeric', 'V3_combined',
       'V4_combined_numeric', 'V4_combined', 'ConfirmedCases',
       'ConfirmedDeaths', 'Stringen

In [14]:
vaccinations[vaccinations["FIPS"] == "26023"]

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
1690,2022-05-21,26023,20,Branch County,MI,95.099998,20728.0,47.599998,20728.0,50.700001,...,8.0,8.0,8.0,7.0,43517,40880,7359,37045,33521,8129
4057,2022-05-20,26023,20,Branch County,MI,95.099998,20725.0,47.599998,20725.0,50.700001,...,8.0,8.0,8.0,7.0,43517,40880,7359,37045,33521,8129
9509,2022-05-19,26023,20,Branch County,MI,95.099998,20723.0,47.599998,20723.0,50.700001,...,8.0,8.0,8.0,7.0,43517,40880,7359,37045,33521,8129
10415,2022-05-18,26023,20,Branch County,MI,95.099998,20723.0,47.599998,20723.0,50.700001,...,8.0,8.0,8.0,7.0,43517,40880,7359,37045,33521,8129
13584,2022-05-17,26023,20,Branch County,MI,95.099998,20721.0,47.599998,20721.0,50.700001,...,8.0,8.0,8.0,7.0,43517,40880,7359,37045,33521,8129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707103,2020-12-17,26023,51,Branch County,MI,0.0,0.0,0.0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,43517,<NA>,<NA>,37045,33521,<NA>
1710448,2020-12-16,26023,51,Branch County,MI,0.0,0.0,0.0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,43517,<NA>,<NA>,37045,33521,<NA>
1714729,2020-12-15,26023,51,Branch County,MI,0.0,0.0,0.0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,43517,<NA>,<NA>,37045,33521,<NA>
1716719,2020-12-14,26023,51,Branch County,MI,0.0,0.0,0.0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,43517,<NA>,<NA>,37045,33521,<NA>


In [2]:
vaccinations = Vaccinations().load()

In [3]:
vaccinations.columns

Index(['Date', 'FIPS', 'MMWR_week', 'Recip_County', 'Recip_State',
       'Completeness_pct', 'Administered_Dose1_Recip',
       'Administered_Dose1_Pop_Pct', 'Administered_Dose1_Recip_5Plus',
       'Administered_Dose1_Recip_5PlusPop_Pct',
       'Administered_Dose1_Recip_12Plus',
       'Administered_Dose1_Recip_12PlusPop_Pct',
       'Administered_Dose1_Recip_18Plus',
       'Administered_Dose1_Recip_18PlusPop_Pct',
       'Administered_Dose1_Recip_65Plus',
       'Administered_Dose1_Recip_65PlusPop_Pct', 'Series_Complete_Yes',
       'Series_Complete_Pop_Pct', 'Series_Complete_5Plus',
       'Series_Complete_5PlusPop_Pct', 'Series_Complete_5to17',
       'Series_Complete_5to17Pop_Pct', 'Series_Complete_12Plus',
       'Series_Complete_12PlusPop_Pct', 'Series_Complete_18Plus',
       'Series_Complete_18PlusPop_Pct', 'Series_Complete_65Plus',
       'Series_Complete_65PlusPop_Pct', 'Booster_Doses',
       'Booster_Doses_Vax_Pct', 'Booster_Doses_12Plus',
       'Booster_Doses_12Plus_V

In [10]:
vaccinations = Vaccinations().load()

,FIPS,SVI_CTGY
0,02090,B
1,UNK,NaN
2,01015,D
3,05087,C
4,06087,C
...,...,...
1723084,27007,D
1723085,55105,B
1723086,29017,B
1723087,48347,D


In [18]:
vaccinations.columns

Index(['Date', 'FIPS', 'MMWR_week', 'Recip_County', 'Recip_State',
       'Completeness_pct', 'Administered_Dose1_Recip',
       'Administered_Dose1_Pop_Pct', 'Administered_Dose1_Recip_5Plus',
       'Administered_Dose1_Recip_5PlusPop_Pct',
       'Administered_Dose1_Recip_12Plus',
       'Administered_Dose1_Recip_12PlusPop_Pct',
       'Administered_Dose1_Recip_18Plus',
       'Administered_Dose1_Recip_18PlusPop_Pct',
       'Administered_Dose1_Recip_65Plus',
       'Administered_Dose1_Recip_65PlusPop_Pct', 'Series_Complete_Yes',
       'Series_Complete_Pop_Pct', 'Series_Complete_5Plus',
       'Series_Complete_5PlusPop_Pct', 'Series_Complete_5to17',
       'Series_Complete_5to17Pop_Pct', 'Series_Complete_12Plus',
       'Series_Complete_12PlusPop_Pct', 'Series_Complete_18Plus',
       'Series_Complete_18PlusPop_Pct', 'Series_Complete_65Plus',
       'Series_Complete_65PlusPop_Pct', 'Booster_Doses',
       'Booster_Doses_Vax_Pct', 'Booster_Doses_12Plus',
       'Booster_Doses_12Plus_V

3225

In [16]:
vaccinations

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,2022-05-21,02090,20,Fairbanks North Star Borough,AK,97.5,66199.0,68.400002,66169.0,73.800003,...,2.0,2.0,2.0,3.0,96849,89670,15825,80563,73845,10828
1,2022-05-21,UNK,20,Unknown County,AK,97.5,15439.0,<NA>,15424.0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2022-05-21,01015,20,Calhoun County,AL,92.5,65339.0,57.5,65336.0,61.0,...,2.0,2.0,2.0,2.0,113605,107037,17905,97757,89132,20547
3,2022-05-21,05087,20,Madison County,AR,90.5,8382.0,50.599998,8381.0,54.0,...,3.0,3.0,3.0,2.0,16576,15527,2800,14093,12727,3211
4,2022-05-21,06087,20,Santa Cruz County,CA,97.599998,236526.0,86.599998,236214.0,90.900002,...,4.0,4.0,4.0,3.0,273213,259767,38453,239473,221314,47217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723084,2020-12-13,27007,51,Beltrami County,MN,0.0,0.0,0.0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,47188,<NA>,<NA>,39001,35342,<NA>
1723085,2020-12-13,55105,51,Rock County,WI,0.0,0.0,0.0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,163354,<NA>,<NA>,139259,125952,<NA>
1723086,2020-12-13,29017,51,Bollinger County,MO,0.0,0.0,0.0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,12133,<NA>,<NA>,10467,9561,<NA>
1723087,2020-12-13,48347,51,Nacogdoches County,TX,0.0,0.0,0.0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,65204,<NA>,<NA>,55079,50203,<NA>


In [17]:
vaccinations[["SVI_CTGY"]].drop_duplicates()

,SVI_CTGY
0,B
1,NaN
2,D
3,C
7,A
